### Import libraries

In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
import unidecode
import re
import os

### Upload preclean Data Frame and Keywords file.

In [2]:
# 1) Upload CSV file containing scraped clean data (data_cleaning.ipynb) and upload keywords for filter 1 and 2.

# Specifies directory path where the Excel files are located.
directory_ads = r'C:\Users\DAV\Documents\Python\Python_Project\House_price_project\data\raw'  

directory_keywords = r'C:\Users\DAV\Documents\Python\Python_Project\House_price_project'  

# Define files name.
ads_file = 'pre-clean.csv'  
keywords_file = 'key_words.xlsx'  

# Build full file paths.
ads_file_path = os.path.join(directory_ads, ads_file)
keywords_file_path = os.path.join(directory_keywords, keywords_file)

# Load ad data.
df = pd.read_csv(ads_file_path, dtype=str)

# Load keyword data.
xls = pd.ExcelFile(keywords_file_path)

# Load each sheet into a separate DataFrame (filter 1 and 2).
keywords_df = pd.read_excel(xls, 'keywords')
municipios_filter_2_df = pd.read_excel(xls, 'municipios_filter_2')

# Convert municipalities column to a list.
municipios_filter_2 = municipios_filter_2_df['municipio'].tolist()

In [3]:
# 2) Add empty columns for keyword, municipality and zone.
df['keyword'] = None
df['municipality'] = None
df['zone'] = None

### First Filter Function

In [4]:
# 3) First Filter Function. 
def filter_1(ads_df, keywords_df):

# First filter: Search for matches using keyword list.
    for index, row in ads_df.iterrows():
        title = row['title_normalized']
        for _, keyword_row in keywords_df.iterrows():
            keyword = keyword_row['keyword']
            # Use fuzzy matching but check for an exact match on the keyword.
            if keyword in title:
                # Only takes keyword if it is found as a full term.
                term_pattern = r'\b' + re.escape(keyword) + r'\b'
                if re.search(term_pattern, title):
                    ads_df.at[index, 'keyword'] = keyword_row['keyword']
                    ads_df.at[index, 'municipality'] = keyword_row['municipality']
                    # Specific change: Convert zone to text without decimals.
                    if pd.notna(keyword_row['zone']):
                        ads_df.at[index, 'zone'] = str(int(float(keyword_row['zone'])))

                    break  # Exit loop if we find a match.

    # Make sure only ads with keywords go to the result.
    filter_1_results = ads_df[ads_df['keyword'].notnull()]

# Run filter 1.
filter_1(df, keywords_df)

### Second Filter Function

In [5]:
# 4) Second Filter Function
def filter_2(ads_df, municipality_filter_2):
    # Ads filter: found no matches in filter 1 (keyword = None).
    without_keyword_df = ads_df[ads_df['keyword'].isnull()].copy()

# Second filter: apply filter using municipalities.
    for index, row in without_keyword_df.iterrows():
        found_municipality = None

        # Search for a match with municipality.
        for municipality in municipality_filter_2:
            similarity = fuzz.token_set_ratio(row['title_normalized'], municipality)
            if similarity >= 85:
                found_municipality = municipality
                ads_df.at[index, 'municipality'] = found_municipality
                break

        # Detect zones.
        found_zone_search = re.compile(r'\b(?:zona\s*(\d{1,2})|z\.\s*(\d{1,2})|z\s*(\d{1,2}))\b', re.IGNORECASE)
        zones_found = found_zone_search.findall(row['title_normalized'])

        if zones_found:
            found_zone = zones_found[0][0] or zones_found[0][1] or zones_found[0][2]
            ads_df.at[index, 'zone'] = f"{found_zone}"

            # Specific allocation for Guatemala City (only when area is mentioned).
            if not found_municipality:
                ads_df.at[index, 'municipality'] = 'guatemala city'

        # Assign keyword if municipality was found.
        if found_municipality:
            ads_df.at[index, 'keyword'] = found_municipality  # Usar el municipio como keyword
        elif not found_municipality and zones_found:
            ads_df.at[index, 'keyword'] = 'guatemala city'

    # Filter the first and second filter results. 
    filter_2_results = ads_df[ads_df['keyword'].notnull() | ads_df['municipality'].notnull()]

# Run filter 2.
filter_2(df, municipios_filter_2)

### Third Filter Function

In [6]:
# 5) Ads filter that didn't find matches in filters 1 and 2 (municipality = None).
without_keyword_municipality_df = df[(df['keyword'].isnull()) & (df['municipality'].isnull())].copy()

# Specific keywords related to municipalities on Carretera a El Salvador.
carretera_el_salvador_keywords = ["carretera a el salvador", "caes", "c.a.e.s.", "ces", "csalvador", "c salvador", "ruta a el salvador", "carretera salvador", "carretera el salvador"]

# Evaluate ads for filter 3.
for index, row in without_keyword_municipality_df.iterrows():
    found_keyword = None
    found_municipality = None

    # Check if a 'km number' exists.
    km_match = re.search(r'km\s*([\d.]+)', row['title_normalized'], re.IGNORECASE)
    if km_match:
        km_value = float(km_match.group(1))
        if km_value < 16:
            found_municipality = 'santa catarina pinula'
        elif 16 <= km_value < 18:
            found_municipality = 'fraijanes'
        elif 18 <= km_value < 24:
            found_municipality = 'san jose pinula'
        elif km_value >= 24:
            found_municipality = 'villa canales'

    # Check match for specific keywords in title.
    for keyword in carretera_el_salvador_keywords:
        if keyword in row['title_normalized']:
            found_keyword = 'carretera a el salvador'
            break
    
    # Verify coincidence with municipalities.
    for municipio in ["santa catarina pinula", "fraijanes", "san jose pinula", "villa canales"]:
        if municipio in row['title_normalized']:
            found_keyword = municipio
            found_municipality = municipio

    # Assign and mark results
    if found_municipality and found_keyword == None:
        found_keyword = found_municipality
    
    df.at[index, 'keyword'] = found_keyword if found_keyword else 'unknown'
    df.at[index, 'municipality'] = found_municipality if found_municipality else 'unknown'

# Filter final: filter results and those without keyword or municipality.
filter_3_results = df[(df['keyword'] != 'unknown') | (df['municipality'] != 'unknown')]
without_keyword_final = df[(df['keyword'] == 'unknown') & (df['municipality'].isnull())]

### Determine Property Type (House or Apartment)

In [7]:
# 6) Keyword lists to identify houses and apartments.
house_keywords = ['casa', 'vivienda', 'chalet', 'townhouse']
apartment_keywords = ['apartamento', 'departamento', 'torre', 'apto', 'loft', 'edificio']

def determine_property_type(title, description):
    # Make sure title and description are text strings.
    title = str(title) if title is not None else ''
    description = str(description) if description is not None else ''

    # Join title and description into a single string.
    text = (title + " " + description).lower()
    
    # Search for house keywords.
    if any(word in text for word in house_keywords):
        return 'house'
    
    # We search for apartment keywords.
    if any(word in text for word in apartment_keywords):
        return 'apartament'
    
    # If no property type is found.
    return 'unknown'

# Reapply function to DataFrame.
df.loc[:,'type'] = df.apply(lambda row: determine_property_type(row['title_normalized'], row['description_normalized']), axis=1)

# Filter rows ('unknown') in 'type' column.
df = df[df['type'] != 'unknown']

### Delete 'unknown' values in 'Municipality' column.

In [8]:
# 7) Delete rows where column 'Municipality' has 'unknown' value.

# Filter rows ('unknown') in 'type' column.
df = df[df['municipality'] != 'unknown']

In [9]:
# 8) Fill 'NaN' values in 'zone' with 'Not Specified'
df['zone'] = df['zone'].fillna('not specified')

### Save Clean Data Frame (CSV file)

In [10]:
# 9) Save Clean Data.

# Specify directory path.
directory = 'C:\\Users\\DAV\\Documents\\Python\\Python_Project\\House_price_project\\data\\clean\\'

# Define the file name.
filename = 'clean-data.csv'

# Build full file path.
output_path = directory + filename

# Save DataFrame to an CSV file in the specified directory.
df.to_csv(output_path, index=False)